In [1]:
#-*-encoding:utf-8-*-
import requests
from bs4  import BeautifulSoup
import random
import json
import time

def cn_Login(r,isProxy=0,proxyIp='7.175.57.56',proxyPort='8083'):
    if isProxy==0:
        proxy={}
    elif(isProxy==1):
        proxy={'http':proxyIp+':'+proxyPort,'https':proxyIp+':'+proxyPort}
    
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'en-US,en;q=0.5','Accept-Encoding':'gzip, deflate'}

    sendsms1='41608446240A6782F2A0F031426EDC066CF24674F3F0586A4D5FF056D75FF4AB'
    sendsms2='41608446240A6782F2A0F031426EDC066CF24674F3F0586AF1E3983438A09296FFFB9CF36AA16F647C256D62346762770768A4F7DFE431F4'
    userInfo=sendsms2

    
    url='https://4acasp.gmcc.net///auth/nextlogin.jsp?target=https://4a.gmcc.net/first.do?method=login&appCode=IAM000'
    import time
    result1=r.get(url,headers=header,proxies=proxy,verify=False)
 
    #获取random_form
    soup1=BeautifulSoup(result1.text,'lxml')
    loginForm1=str(soup1.select('#loginForm')[0].select('input')[0]).split('=')[3].replace('"','').replace('/>','')

    #登陆界面
    url='https://4acasp.gmcc.net/jk.do?method=checkUserType&userId=shizhongxia'
    r.post(url,data={'method':'checkUserType','userId':'shizhongxia'},headers=header,proxies=proxy,verify=False)

    #-----应该返回：SUCCESSG|TshizhongxiaG|T------
    # print s.text
    url='https://4acasp.gmcc.net/loginForward.do?target=https://4a.gmcc.net/first.do?method=login&appCode=IAM000&sendsms='+sendsms1
    parma={'random_form':loginForm1,'loginPage':'/auth/nextlogin.jsp','smsNameText':'shizhongxia','smsName':'shizhongxia'}
    result2=r.post(url,data=parma,headers=header,proxies=proxy,verify=False)
    soup2=BeautifulSoup(result2.text,'lxml')
    loginForm2=str(soup2.select('#loginForm')[0].select('input')[0]).split('=')[3].replace('"','').replace('/>','')
    parma={'sendsms':sendsms2}
    s=r.post('https://4acasp.gmcc.net/jk.do?method=checkSMSUser',data=parma,headers=header,proxies=proxy,verify=False)
    url='https://4acasp.gmcc.net/jk.do?method=checkSMSJK&target=https://4a.gmcc.net/first.do?method=login&authType=noteAuth&appCode=IAM000&loginPage=/auth/nextlogin.jsp&userInfo='+userInfo
    r.post(url,headers=header,proxies=proxy,verify=False)
    url='https://4acasp.gmcc.net/sendSms.do?smsfrom=send&target=https://4a.gmcc.net/first.do?method=login&authType=noteAuth&appCode=IAM000&loginPage=/auth/nextlogin.jsp'

    #下面这一步获取短信验证码
    r.post(url,data=parma,headers=header,proxies=proxy,verify=False) 
    print u'已经发送短信验证码'
    smsinput=raw_input(u'验证码:')
    print u'正在登陆，请稍后...'

    # 验证
    url='https://4acasp.gmcc.net/loginServlet.do?target=https://4a.gmcc.net/first.do?method=login&authType=noteAuth&appCode=IAM000'
    parma={'random_form':loginForm2,'loginPage':'/auth/nextUserLogin.jsp','userName':'shizhongxia','passWord':'','tokenName':'shizhongxia','isUsePin':'true','updatePin':'false','tokenpassword':'','pinCode':'','smsName':'shizhongxia','smsPwd':'1234qweR','dynamic_smsPwd':smsinput,'checkCodeTemp':'','figerName':'','challengeName':'shizhongxia','challegePassword':'','vpn':''}
    result=r.post(url,data=parma,headers=header,proxies=proxy,verify=False)

    #信息同步
    random2='0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))
    s=r.get('https://4a.gmcc.net/synAuthz.do?method=syn&rnd='+random2,headers=header,proxies=proxy,verify=False)
    #-----{success:true,code:'null',text:'null'}---------

    #登陆4A
    random2='0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))
    r.get('https://4a.gmcc.net/main.jsp?rnd='+random2,headers=header,proxies=proxy,verify=False)
    r.get('https://4a.gmcc.net/SystemmsgPortlet.lp?id=systemmsg&userID=shizhongxia&resType=1',headers=header,proxies=proxy,verify=False)
    random2='0.'+str(random.randint(10000000000000000,99999999999999999))+str(random.randint(1,9))
    r.get('https://4a.gmcc.net/synAuthz.do?method=syn2AppServer&rnd='+random2,headers=header,proxies=proxy,verify=False)
    r.get('https://4a.gmcc.net/page/resgroup/searchFrame.jsp',headers=header,proxies=proxy)
    r.post('https://4a.gmcc.net/keystore.do',data={'method':'getKeyStoreList'},headers=header,proxies=proxy,verify=False)
    #get:https://4a.gmcc.net/page/resource/appres/appresourceQuery.do?method=listIni
    r.post('https://4a.gmcc.net/page/resource/appres/appresourceQuery.do',data={'method':'pageQuery','_gt_json':'{"recordType":"object","pageInfo":{"pageSize":50,"pageNum":1,"totalRowNum":0,"totalPageNum":0,"startRowNum":0,"endRowNum":0},"columnInfo":[{"id":"id","header":"资源ID","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"hasDueAcc","header":"hasDueAcc","fieldName":"hasDueAcc","fieldIndex":"hasDueAcc","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"groupName","header":"资源组","fieldName":"groupName","fieldIndex":"groupName","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"name","header":"资源名称","fieldName":"name","fieldIndex":"name","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"ip","header":"IP地址","fieldName":"ip","fieldIndex":"ip","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"restype","header":"资源类型","fieldName":"restype","fieldIndex":"restype","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"appResAccountList","header":"从帐号列表","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"ssoMode","header":"ssoMode","fieldName":"ssoMode","fieldIndex":"ssoMode","sortOrder":null,"hidden":"true","exportable":true,"printable":true},{"id":"sysResClientList","header":"登录","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"opt","header":"操作","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true}],"sortInfo":[],"filterInfo":[],"parameters":{},"action":"load"}'},headers=header,proxies=proxy,verify=False)
    r.post('https://4a.gmcc.net/keystore.do',data={'method':'getKeyStoreList'},headers=header,proxies=proxy,verify=False)

    #点击ESOP登陆
    params2={'method':'pageQuery','_gt_json':'{"recordType":"object","pageInfo":{"totalRowNum":-1,"pageSize":50,"pageNum":1,"totalPageNum":0,"startRowNum":0,"endRowNum":0},"columnInfo":[{"id":"id","header":"资源ID","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"hasDueAcc","header":"hasDueAcc","fieldName":"hasDueAcc","fieldIndex":"hasDueAcc","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"groupName","header":"资源组","fieldName":"groupName","fieldIndex":"groupName","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"name","header":"资源名称","fieldName":"name","fieldIndex":"name","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"ip","header":"IP地址","fieldName":"ip","fieldIndex":"ip","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"restype","header":"资源类型","fieldName":"restype","fieldIndex":"restype","sortOrder":null,"hidden":true,"exportable":true,"printable":true},{"id":"appResAccountList","header":"从帐号列表","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"ssoMode","header":"ssoMode","fieldName":"ssoMode","fieldIndex":"ssoMode","sortOrder":null,"hidden":"true","exportable":true,"printable":true},{"id":"sysResClientList","header":"登录","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true},{"id":"opt","header":"操作","fieldName":"id","fieldIndex":"id","sortOrder":null,"hidden":false,"exportable":true,"printable":true}],"sortInfo":[],"filterInfo":[],"parameters":{},"action":"load"}','queryName':'','queryIP':'','isQuerySubTree':'1'}
    result2=r.post('https://4a.gmcc.net/page/resource/appres/appresourceQuery.do',data=params2,headers=header,proxies=proxy,verify=False)
    json2=json.loads(result2.text)
    appResAccountList=''
    appResAccountList=json2.get('data')[1].get('appResAccountList').split('|')[0][:-1]
    id_num=json2.get('data')[1].get('id')           

    #同步数据
    r.post('https://4a.gmcc.net/keystore.do',data={'method':'getKeyStoreList'},headers=header,proxies=proxy,verify=False)
    r.post('https://4a.gmcc.net/page/resource/resourceQuery.do?',data={'method':'isAuthDateValid','subaccName':'AGZGT0000829'},headers=header,proxies=proxy,verify=False)
    params2={'method':'checkPolicy4JK','resId':id_num,'resAccId':appResAccountList,'date':str(time.time()).replace('.','')+str(random.randint(1,9))}
    result2=r.post('https://4a.gmcc.net/page/jk/sso/appJKLogin.do',data=params2,headers=header,proxies=proxy)
    params2={'method':'queryLoginJKStatus','resId':id_num,'resAccId':appResAccountList,'date':str(time.time()).replace('.','')+str(random.randint(1,9))}
    r.post('https://4a.gmcc.net/page/jk/sso/appJKLogin.do',data=params2,headers=header,proxies=proxy,verify=False)
    params2={'method':'wantGetMACAddress','resId':id_num}
    r.post('https://4a.gmcc.net/sso.do',data=params2,headers=header,proxies=proxy,verify=False)
    url2='https://4a.gmcc.net/sso.do?method=appssoData&accID='+appResAccountList+'&resID='+id_num+'&softname=webAppByIE&date='+str(time.time()).replace('.','')+str(random.randint(1,9))
    print '-----------测试是否登陆ESOP成功-----------'
    print r.get(url2,headers=header,proxies=proxy,verify=False)
    print ''


In [ ]:
import requests

r=requests.session()
cn_Login(r,0)

/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.org/en/latest/security.html
  InsecureRequestWarning)
/usr/local/lib/python2.7/dist-packages/requests/packages/urllib3/connectionpool.py:791: InsecureRequestWarning: Unverified H

已经发送短信验证码


In [4]:
# proxy={'http':'7.175.57.56:8083','https':'7.175.57.56:8083'}
# 思路：第一步导出产品编码和账户编码，然后套用连接查询


# 查询集团列表
def cn_ZongHeChaXun_XuLie(r,proxy,cn_input,cn_type=1,isPrint=1,cn_city='广州'):
    #cn_type={0:产品编码,1:集团名称}
    cn_type_grpName=''
    cn_type_productNum=''
    if(cn_type==0):
        cn_type_grpName=''
        cn_type_productNum=cn_input
    elif(cn_type==1):
        cn_type_grpName='%'+cn_input+'%'
        cn_type_productNum=''
    
    #序列字典
    dic={'湛江':'759','茂名':'668','肇庆':'758','广州':'200','深圳':'755','东莞':'769','佛山':'757','汕头':'754','珠海':'756','惠州':'752','中山':'760','揭阳':'663','江门':'750','韶关':'751','河源':'762','梅州':'753','潮州':'768','云浮':'766','清远':'763','汕尾':'660','阳江':'662'}
    cn_city_num=dic.get(cn_city)
    
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'en-US,en;q=0.5','Accept-Encoding':'gzip, deflate'}
    params={'queryConditionBean.comeFromOrderTrace':'','queryConditionBean.grpprodId':'','queryConditionBean.queryMode':'group','queryConditionBean.haveQueryed':'','queryConditionBean.regionID':'','queryConditionBean.qryRgeion':cn_city_num,'queryMode':'group','queryConditionBean.virtualCode':'','queryConditionBean.grpName':cn_type_grpName,'queryConditionBean.productNum':cn_type_productNum,'queryConditionBean.custManCode':'','queryConditionBean.portNum':'','queryConditionBean.feeAndConnPortNum':'','queryConditionBean.mobileExchangeNum':'','queryConditionBean.gn4001Num':'','queryConditionBean.startDate':'','queryConditionBean.startDate_format':'yyyy-MM-dd','queryConditionBean.endDate':'','queryConditionBean.endDate_format':'yyyy-MM-dd','queryConditionBean.bBossOrderNum':'','queryConditionBean.servNum':'','queryConditionBean.netCode':'','queryConditionBean.homeNumber':'','tab1':'','tab2':'','tab3':''}
    url='http://ngesop.boss.gmcc.net/ngcustcare_fs/group/groupQuery/queryGroupCust.action'
    
    #第一个连接失败，一般都是没有登录
    try:
        result=r.post(url,data=params,headers=header,proxies=proxy)
    except:
        print '没有登录4A'
        exit()
        
    soup=BeautifulSoup(result.text,'lxml')
    table=soup.select('#columnDiv2')[0].select('#mainTable')[0].select('#groupListTd')[0].select('#custTable')[0].select('td')
    temp1=''
    
    if len(table)==1:
        n=str(table[0]).find('title=')
        temp1=str(table[0])[n+7:n+17]
        #判断是打印还是给其他函数调用
        if(isPrint==1):
            print ''.join(temp1.split()),''.join(table[0].text.split())
            print ''
        elif(isPrint==0):
            return ''.join(temp1.split())
    else:
        for a in table:
            n=str(a).find('title=')
            print str(a)[n+7:n+17],a.text   
        print ''   

        
#锁定查询的集团,输入的是 集团编码
def cn_ZongHeChaXun_XuanZe(r,proxy,cn_input):
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'en-US,en;q=0.5','Accept-Encoding':'gzip, deflate'}
    url='http://ngesop.boss.gmcc.net/ngcustcare_fs/group/groupQuery/queryGroupByVipCode.action?virtualCode='+cn_input+'&regionID=&gn4001Num=&bBossOrderNum='
    params={'virtualCode':cn_input,'regionID':'','gn4001Num':'','bBossOrderNum':''}
    #第一个连接失败，一般都是没有登录
    try:
        r.post(url,data=params,headers=header,proxies=proxy)
    except:
        print '没有登录4A'
        exit()
        

        
# 查询集团概况
def cn_ZongHeChaXun_GaiKuang(r,proxy,cn_input):
    #先选择集团
    cn_ZongHeChaXun_XuanZe(r,proxy,cn_input)
    #再查询
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'en-US,en;q=0.5','Accept-Encoding':'gzip, deflate'}
    url='http://ngesop.boss.gmcc.net/ngcustcare_fs/group/groupQuery/custFrameAction.action?ran=0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))
    
    #第一个连接失败，一般都是没有登录
    try:
        result=r.get(url,headers=header,proxies=proxy)
    except:
        print '没有登录4A'
        exit()
    
    soup=BeautifulSoup(result.text,'lxml')
    temp=soup.select('.ballistic_bg')[0].select('td')
    print u'----集团概况----'
    print ''.join(temp[3].text.split())#集团编码
    print ''.join(temp[1].text.split())#集团名称
    print ''.join(temp[29].text.split())#集团归属
    print ''.join(temp[17].text.split()),''.join(temp[19].text.split())#客户经理
    print ''.join(temp[21].text.split())#集团分层分级
    print ''

    
    
#查集团产品编码，输入的是 集团编码|产品编码
def cn_ZongHeChaXun_ChanPin(r,proxy,cn_input,cn_type=0,cn_state=9,isPrint=1,cn_city='广州'): 
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'en-US,en;q=0.5','Accept-Encoding':'gzip, deflate'}
    #如果需要打印为1，如果是用于其他函数调用的为0
    if isPrint==1:
        print u'----产品编码明细----'
    else:
        cn_dict={}
        j=0
   
    #cn_type={0:产品编码,1:集团编码}
    if cn_type==0:
        cn_temp_cusNum=cn_ZongHeChaXun_XuLie(r,proxy,cn_input,0,0,cn_city)
        cn_ZongHeChaXun_XuanZe(r,proxy,cn_temp_cusNum)
        url='http://ngesop.boss.gmcc.net/ngcustcare_fs/group/groupQuery/subsFrameAction.action?ran='+'0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))+'&comeFromOrderTrace=&grpprodId=&virtualCode=&productCode='+cn_input
    elif cn_type==1:
        cn_ZongHeChaXun_XuanZe(r,proxy,cn_input)
        url='http://ngesop.boss.gmcc.net/ngcustcare_fs/group/groupQuery/subsFrameAction.action?ran='+'0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9))+'&comeFromOrderTrace=&grpprodId=&virtualCode='+cn_input+'&productCode='
 
    result=r.get(url,headers=header,proxies=proxy)
    soup=BeautifulSoup(result.text,'lxml')
    content=soup.select('#subsListTable')[0].select('.list_title')
    cn_len=xrange(0,len(content),5)
    #状态： all=9|正使用=1|集团产品停用=2|欠费销户=4|其他=8
    cn_state_result={1:(u'正使用'),2:(u'集团产品停用'),3:(u'正使用',u'集团产品停用'),4:(u'欠费销户'),5:(u'正使用',u'欠费销户'),6:(u'集团产品停用',u'欠费销户'),7:(u'正使用',u'集团产品停用',u'欠费销户'),8:(u'集团预约销户',u'正式销户')}
    for i in cn_len:
        temp4=''.join(content[i+4].text.split())
        #判断状态
        if(cn_state==9):
            pass
        elif(temp4 in cn_state_result.get(cn_state)):
            pass
        elif(cn_state==8):
            pass
        else:
            continue    
       
        temp1=''.join(content[i+1].text.split())
        temp2=''.join(content[i+2].text.split())
        temp3=''.join(content[i+3].text.split())
        temp5=''.join(content[i].text.split())
        if isPrint==1:
            print i,temp1,temp3,temp4
        else:
            cn_list=[]
            cn_list.append(temp3)#产品编码
            cn_list.append(temp2)#账号编码
            cn_list.append(temp1)#产品名称
            cn_list.append(temp4)#产品状态
            cn_list.append(temp5)#用户编码
            cn_dict[j]=cn_list
            j+=1
    if isPrint==1:
        print '----END----'
        print ''
    else:
        return cn_dict #输出为一个字典{id:[产品编码，账号编码，产品名称，产品状态,用户编码]}

    
#查询产品编码的详情,输入的为产品编码数组
def cn_ZongHeChaXun_ChanPin_Detail(r,proxy,cn_input,detalLevel=1):
    #不知道是否需要先查询序列？？？？
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'en-US,en;q=0.5','Accept-Encoding':'gzip, deflate'}
   
    for i in xrange(len(cn_input)):
        #查询产品信息
        print u'----产品信息----'
        try:
            result=r.get('http://ngesop.boss.gmcc.net/ngcustcare_fs/group/groupQuery/subsInfoAction.action?subId='+cn_input[i][4]+'&servNumber='+cn_input[i][0]+'&ran=0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9)))           
        except:
            print '没有登录4A'
            exit()
        soup=BeautifulSoup(result.text,'lxml')
        cn_result_temp=soup.select('#theTable')[0].select('tr')
        for j in cn_result_temp:
            print '  '.join(j.text.split())
        print ''
        print u'----联系人信息----'
        #查看联系人信息
        result=r.get('http://ngesop.boss.gmcc.net/ngcustcare_fs/group/groupQuery/subsLinkInfoAction.action?subId='+cn_input[i][4]+'&ran=0.'+str(random.randint(100000000000000,999999999999999))+str(random.randint(1,9)))
        soup=BeautifulSoup(result.text,'lxml')
        cn_result_temp=soup.select('#tb1')[0].select('tr')
        for j in cn_result_temp:
            print '  '.join(j.text.split())
        print ''
        print u'----产品列表----'
        #查询产品列表
        result=r.get('http://ngesop.boss.gmcc.net/ngcustcare_fs/group/groupQuery/subsProductAction.action?queryHistory=true&subId='+cn_input[i][4]+'&ran=0.'+str(random.randint(10000000000000000,99999999999999999))+str(random.randint(1,9)))
        soup=BeautifulSoup(result.text,'lxml')
        cn_result_temp=soup.select('.ballistic_bg')[0].select('#subsProdiucForm')[0].select('#tabMain3')[0].select('tr')
        for j in cn_result_temp:
            print '  '.join(j.text.split())
        print ''
        #查询产品属性
        if(detalLevel==2):
            result=r.get('http://ngesop.boss.gmcc.net/ngcustcare_fs/group/groupQuery/subsAttributeAction.action?subId='+cn_input[i][4]+'&ran=0.'+str(random.randint(1000000000000000,9999999999999999))+str(random.randint(1,9)))
            soup=BeautifulSoup(result.text,'lxml')
            print soup
    print '----END----'
    print ''
        
        
        
#查询集团编码全部产品的余额，输入的为一个字典{id:[产品编码，账号编码，产品名称，产品状态,用户编码]}
def cn_ZhangWuChaXun_YuE(r,proxy,cn_input,simpleORdetail='s',BeginNum=0,cn_city='广州'):
    header={'User-Agent':'Mozilla/5.0 (Windows NT 6.2; WOW64; rv:42.0) Gecko/20100101 Firefox/42.0','Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Language':'en-US,en;q=0.5','Accept-Encoding':'gzip, deflate'}
#     print u'----产品编码余额----'
    #没有测试连接是否可用

    #简单余额查询，仅知道用余额,命令：'s'
    url='http://ngesop.boss.gmcc.net/ngcustcare_fs/chargesrv/accountInfoQry/balanceQry/query.action'
    cn_xrange_temp=xrange(BeginNum,len(cn_input))
    if simpleORdetail=='s':
        for i in cn_xrange_temp:#BeginNum用于批量查询中途断了可以继续查询
            params={'subsId':cn_input[i][4],'searchType':'servNumber','searchNumber':cn_input[i][0],'balanceType':'CanUse','curAcctId':cn_input[i][1]}
            result=r.post(url,data=params,headers=header,proxies=proxy)
            soup=BeautifulSoup(result.text,'lxml')
            
            cn_temp_list=soup.select('#columnDivBalanceQry')[0].select('.table_list03')[0].select('tr')
            cn_temp_list_1=cn_temp_list[3].text.split()
            return cn_temp_list_1[3]
    elif simpleORdetail=='d':
        for i in cn_xrange_temp:#BeginNum用于批量查询中途断了可以继续查询
            params={'subsId':cn_input[i][4],'searchType':'servNumber','searchNumber':cn_input[i][0],'balanceType':'CanUse','curAcctId':cn_input[i][1]}
            result=r.post(url,data=params,headers=header,proxies=proxy)        
            soup=BeautifulSoup(result.text,'lxml')
            cn_temp_list=soup.select('#columnDivBalanceQry')[0].select('.table_list03')[0].select('tr')
            
            print cn_input[i][0],cn_input[i][2],cn_input[i][3]
            print ''
            print '  '.join(cn_temp_list[3].text.split())
            print '  '.join(cn_temp_list[2].text.split())
            print '  '.join(cn_temp_list[4].text.split())
            print '  '.join(cn_temp_list[1].text.split())
            print ''

            url_temp='http://ngesop.boss.gmcc.net/ngcustcare_fs/chargesrv/common/subjectBalance/qrySubjectBalance.action?acctId='+cn_input[i][1]+'&recType=BalanceQry&subjectType=CanUse'
            result=r.get(url_temp,headers=header,proxies=proxy)
            soup=BeautifulSoup(result.text,'lxml')
            cn_result_temp=soup.select('#columnDivSubjectBalance')[0].select('#balanceTable')[0].select('tr')
            cn_temp_xrange=xrange(1,len(cn_result_temp)-1)
            for j in cn_temp_xrange:
                print ' '.join(cn_result_temp[j].text.split())
            print ''

            url_temp='http://ngesop.boss.gmcc.net/ngcustcare_fs/chargesrv/common/bill/qryShouldPayBill.action?acctId='+cn_input[i][1]+'&badDebtType=1'
            result=r.get(url_temp,headers=header,proxies=proxy)
            soup=BeautifulSoup(result.text,'lxml')
            cn_result_temp=soup.select('.ballistic_bg')[0].select('#columnDivchargeBill')[0].select('#billTable')[0].select('tr')
            for i in cn_result_temp:
                if(i.text.split()[0]!=u'帐号'):    
                    print ' '.join(i.text.split())
                else:
#                     print '-------------------------'
                    print ' '.join(i.text.split())
            print ''
            print '====================================================='
            print ''
        print '----END----'   
        print ''



# proxy={'http':'7.175.57.56:8083','https':'7.175.57.56:8083'}

In [6]:
# proxy={'http':'7.175.57.56:8083','https':'7.175.57.56:8083'}
proxy={}

# def cn_ZongHeChaXun_XuLie(r,proxy,cn_input,cn_type=1,isPrint=1,cn_city='广州'):
#cn_type={0:产品编码,1:集团名称}
# cn_ZongHeChaXun_XuLie(r,proxy,'晶东贸易',1,1,'广州')


#def cn_ZongHeChaXun_GaiKuang(r,proxy,cn_input):
# cn_ZongHeChaXun_GaiKuang(r,proxy,'2000126921')  
# 2002521473   2002597183  


# def cn_ZongHeChaXun_ChanPin(r,proxy,cn_input,cn_type=0,cn_state=9,isPrint=1,cn_city='广州'):    
#状态： all=9|正使用=1|集团产品停用=2|欠费销户=4|其他=8
#cn_type={0:产品编码,1:集团编码}
# cn_ZongHeChaXun_ChanPin(r,proxy,'2000444654',1,9,1,'佛山')


#查余额前需要查询产品编码
# def cn_ZhangWuChaXun_YuE(r,proxy,cn_input,simpleORdetail='s',BeginNum=0):
ff=u'20168013476;20118302894;20118200217;20118201416;20118200299;20118200359;20118201091;20118200191;20118303505;20118200965;20118200686;20118201639;20118201875;20118200394;20118201748;20118201264;20118200862;20102594377;30117014743;20102590726;20102040269;20102040907;30117990734;30117011629;30117013823;30117009532;30117013803;30117992335;20102804994;30117993110;20118205328;20118203358;30121009555;30122020924;30122021106;30122021035;30122021075;30122020997;30122020947;30122021083;30122021149;30122021175;30122021107;30122021097;30122021105;30122021109;30122021135;30122021095;30122021069;30122021070;30122021195;30122020962;30122020966;30122020937;30122020944;30122021016;30122021021;30122021222;30122020952;30122020950;30122021014;30122021187;30122021091;30122021017;30122020969;30122021181;30122021289;30122021182;30122021178;30122021027;30122021151;30122020963;30122021028;30122021281;30122021279;30122021278;30122021280;30122021224;30122021026;30122020964;30122021025;30122021031;30122020965;30122021185;30122021167;30122021024;30122021153;30122021358;30122021314;30122021022;30122021316;30122021287;30122021315;30122021285;30122020905;30122021286;30122021162;30122021019;30122021161;30122021018;30122021179;30122021163;30122021189;30122021164;30122021165;30122021188;20118201437;20118200212;20118200664;20118200310;20118201711;20118201310;20118201501;20118201709;20118200320;20118200979;20118201915;20118200690;20118201407;20118308140;20118307273;20118309059;20118308187;20118309740;20118204215;20118305182;20118208242;20118205889;20118209939;20118203888;20118207560;20118205134;20118202543;20118206968;20118207079;20118307895;20118308888;20118308987;20118306888;20118308266;20118309684;20118208931;20118207140;20118206234;20118205824;30122003124;30122001728;30122002926;30122004930;30122000232;30122003421;30122004266;30122002626;30122004780;30122004199;30122001009;30122001466;30122000119;30122002083;30122003123;30122001525;30121009517;30121005057;30122001695;30122001461;30122000565;30122003271;30122003857;30122004319;30122000399;30122000758;30122002917;30122004765;30122002216;30122003026;30122000580;30122002981;30122002535;30122003494;30121003055;30122003731;30122000259;30121001699;20110427699;30122000258;30121015619;30121005526;30122002573;30122004348;30121003879;30122002788;30122001667;30122001323;30122001255;30122002782;30122001048;30122002368;30121007418;30122002827;30121017068;30122001100;40184000106;40184000107;20106590250;20105591833;30122000083;30121012859;30121012706;30121018569;30121013660;30122000943'
ff=ff.split(';')
for i in ff:
    try:
        x=cn_ZongHeChaXun_ChanPin(r,proxy,i,0,9,0,'广州')
        print i,cn_ZhangWuChaXun_YuE(r,proxy,x,'s')
        time.sleep(3)
    except:
        pass



#查询产品编码详情，需要输入产品编码数组
# def cn_ZongHeChaXun_ChanPin_Detail(r,proxy,cn_input,detalLevel=1):
# x=cn_ZongHeChaXun_ChanPin(r,proxy,'21423005200',0,9,0,'佛山')
# cn_ZongHeChaXun_ChanPin_Detail(r,proxy,x,2)


20168013476 0.00
20118302894 0.00
20118200217 0.00
20118201416 0.00
20118200299 0.00
20118200359 0.00
20118201091 0.00
20118200191 0.00
20118303505 0.00
20118200965 0.00
20118200686 0.00
20118201639 0.00
20118201875 0.00
20118200394 0.00
20118201748 0.00
20118201264 0.00
20118200862 0.00
20102594377 730.00
30117014743 0.00
20102590726 0.00
20102040269 0.00
20102040907 0.00
30117990734 0.00
30117011629 0.00
30117013823 0.00
30117009532 0.00
30117013803 0.00
30117992335 0.00
20102804994 0.00
30117993110 0.00
20118205328 0.00
20118203358 0.00
30121009555 0.00
30122020924 0.00
30122021106 0.00
30122021035 0.00
30122021075 0.00
30122020997 0.00
30122020947 0.00
30122021083 0.00
30122021149 0.00
30122021175 0.00
30122021107 0.00
30122021097 0.00
30122021105 0.00
30122021109 0.00
30122021135 0.00
30122021095 0.00
30122021069 0.00
30122021070 0.00
30122021195 0.00
30122020962 0.00
30122020966 0.00
30122020937 0.00
30122020944 0.00
30122021016 0.00
30122021021 0.00
30122021222 0.00
30122020952 

In [ ]:
soup=BeautifulSoup(s,'lxml')
a=soup.select('form')[1].select('#servTabs')
b= a[0].select('tr')[0].select('td')[1].select('input')[0]['value']
print b

In [ ]:
from bs4 import BeautifulSoup
soup=BeautifulSoup(s,'lxml')
a=soup.select('form')[1].select('table')

for a_temp in a:
    b=a_temp.select('tr')
    for b_temp in b:
        b_temp_1=b_temp.select('td')
        cn_xrange=xrange(len(b_temp_1))
        cn_dan=[]
        for i in cn_xrange:
            try:
                if(i%2==0):
                    cn_dan.append(' '.join(b_temp_1[i].text.split()))
                else:
                    if(len(b_temp_1[i].select('input'))>0):
                        cn_dan.append(b_temp_1[i].select('input')[0]['value'])
                    elif(len(b_temp_1[i].select('select'))>0):
                        c=b_temp_1[i].select('select')
                        for temp in c:
                            d=temp.select('option')
                            for temp in d :
                                d_temp=str(temp).find('selected')
                                if(d_temp>0):
                                    cn_dan.append(temp.text)
                    else:
                        cn_dan.append(b_temp_1[i].text)
            except:
                pass
        try:
            print cn_dan[0],cn_dan[1],'                                            ',cn_dan[2],cn_dan[3]
           
        except:
            print ''
    print '==========================================='